<a href="https://colab.research.google.com/github/Fahid08/Test01/blob/main/SpamClassBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

from google.colab import drive
drive.mount('/content/drive')


# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the email dataset
file_path = '/content/drive/My Drive/spam.csv'
df = pd.read_csv(file_path)
print(df)

# Preprocess the text data
#remove duplicate rows
df = df.drop_duplicates()

def clean_text(Message):
    Message = re.sub(r'\n', '', Message)   # remove newline characters
    Message = re.sub(r'\d+', '', Message)  # remove digits
    Message = re.sub(r'\@[A-Za-z0-9]+', '', Message)  # remove mentions
    Message = re.sub(r'https?:\/\/\S+', '', Message)  # remove urls
    Message = re.sub(r'#', '', Message)   # remove hashtags
    Message = Message.translate(str.maketrans("", "", string.punctuation))  # remove punctuation
    Message = re.sub(r'\s+', ' ', Message)  # remove extra spaces
    Message = Message.lower()  # convert to lowercase
    return Message

df['Message'] = df['Message'].apply(clean_text)

# Split the dataset into train and test sets
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encode the input data
train_encodings = tokenizer(train_df.Message.to_list(), truncation=True, padding=True)
test_encodings = tokenizer(test_df.Message.to_list(), truncation=True, padding=True)

# Convert the labels to binary values
train_labels = train_df.Category.replace({'ham': 0, 'spam': 1})
test_labels = test_df.Category.replace({'ham': 0, 'spam': 1})

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))

# Build the BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset.shuffle(100).batch(16), epochs=2, batch_size=16, validation_data=test_dataset.shuffle(100).batch(16))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset.batch(16))
print('Test accuracy:', test_acc)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to i

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
258/258 [==============================] - 10463s 40s/step - loss: 0.1070 - accuracy: 0.9668 - val_loss: 0.0417 - val_accuracy: 0.9845
Epoch 2/2
65/65 [==============================] - 344s 5s/step - loss: 0.0348 - accuracy: 0.9893
Test accuracy: 0.9893307685852051


In [3]:
!pip install tensorflow --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
